In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import pystorm
from pystorm.hal import HAL
from pystorm.hal.net_builder import NetBuilder
from pystorm.hal.run_control import RunControl

In [2]:
# set parameters for network
#   number of neurons
# create the network
#   set tap points - this can be encapsulated and improved later

# what should I set the gain and bias bits to?
# 
X = 8
Y = 8
NNEURON = X*Y
DIM = 1
FMAX = 1000

hal = pystorm.hal.HAL()

In [6]:
net_builder = NetBuilder(hal)

bad_syn = hal.get_calibration("synapse", "high_bias_magnitude")
SX = X // 2
SY = Y // 2
tap_matrix_syn = net_builder.create_default_yx_taps(Y, X, DIM, bad_syn)
tap_matrix = net_builder.syn_taps_to_nrn_taps(tap_matrix_syn)
net = net_builder.create_single_pool_net(Y, X, tap_matrix)
run_control = RunControl(hal, net)

hal.map(net)

AssertionError: 

In [3]:
# collect data for training
# calculate decode weights
#   use linear function offset at 0

total_train_time = 1
total_train_points = 11
offset_time = 0.5

train_time_ns = (np.linspace(offset_time, total_train_time+offset_time, total_train_points)*1E9).astype(int)
train_rates = (FMAX * np.linspace(-1, 1, total_train_points)).reshape((-1, 1))


input_obj = net.inputs[0]
input_vals = {input_obj:(train_time_ns, train_rates)}
outputs, spikes = run_control.run_input_sweep(input_vals)
print(outputs)
print(spikes)

(None, None)
({Pool p1: array([], shape=(0, 64), dtype=uint32)}, array([11825910000, 11826910000, 11827910000, ..., 12933910000,
       12934910000, 12935910000], dtype=uint64))


In [2]:
# run tests
#  deliver an input of 0 